In [2]:
from gensim.models import KeyedVectors
wv = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True, limit=1000000)
wv.save_word2vec_format('vectors.csv')

In [17]:
# read vectors.csv into dictionary
import csv
reader = csv.reader(open('vectors.csv', 'r'))
next(reader)

word2vec = {}
for idx, row in enumerate(reader):
    word = row[0]
    try:
        word2vec[word.split()[0]] = list(map(float, word.split()[1:]))
    except IndexError:
        print(len(word.strip()))
        continue

0
0
0
0
0
0
0
0
0
0
0
0


In [48]:
import pandas as pd
import time

# read in the data
df = pd.read_csv('phrases.csv', encoding = "ISO-8859-1")
# to list
phrases = df['Phrases'].tolist()

In [45]:
st_time = time.time()

phrase2vec = {}
for phrase in phrases:
    # calculate the vector for each phrase given the vectors of the words
    phrase_vector = [0]*300
    words = phrase.split()
    for word in words:
        try:
            phrase_vector = [x + y for x, y in zip(phrase_vector, word2vec[word])]
        except KeyError:
            continue
    phrase_vector = [x/len(words) for x in phrase_vector]
    phrase2vec[phrase] = phrase_vector


# create the matrix of similarity of phrases to each other using cosine similarity
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = np.zeros((len(phrases), len(phrases)))
for i in range(len(phrases)):
    for j in range(len(phrases)):
        similarity_matrix[i][j] = cosine_similarity([phrase2vec[phrases[i]]], [phrase2vec[phrases[j]]])

# get phrases that are similar to each other
similar_phrases = {}
for i in range(len(phrases)):
    similar_phrases[phrases[i]] = []
    for j in range(len(phrases)):
        if similarity_matrix[i][j] > 0.6 and i != j:
            similar_phrases[phrases[i]].append(phrases[j])
end_time = time.time()
print("TIME:", end_time - st_time)

/var/folders/by/kkvcx14x7kj7895wl76s3tmw0000gn/T/ipykernel_47016/3387851597.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  similarity_matrix[i][j] = cosine_similarity([phrase2vec[phrases[i]]], [phrase2vec[phrases[j]]])
/var/folders/by/kkvcx14x7kj7895wl76s3tmw0000gn/T/ipykernel_47016/3387851597.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  similarity_matrix[i][j] = cosine_similarity([phrase2vec[phrases[i]]], [phrase2vec[phrases[j]]])
/var/folders/by/kkvcx14x7kj7895wl76s3tmw0000gn/T/ipykernel_47016/3387851597.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ens

TIME: 0.3759799003601074


In [62]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
st_time = time.time()
# Assuming `phrases` is a list of phrases and `word2vec` is a preloaded word vector dictionary
phrase_vectors = []

# Precompute phrase vectors using numpy for efficient computation
for phrase in phrases:
    words = phrase.split()
    word_vectors = np.array([word2vec[word] for word in words if word in word2vec])
    
    if len(word_vectors) > 0:  # Avoid division by zero
        phrase_vector = np.mean(word_vectors, axis=0)
    else:
        phrase_vector = np.zeros(300)  # Assuming word vectors are of size 300
    
    phrase_vectors.append(phrase_vector)

# Convert list of vectors to a numpy matrix for efficient similarity computation
phrase_vectors_matrix = np.array(phrase_vectors)

# Compute cosine similarity in batches (or all at once if memory allows)
similarity_matrix = cosine_similarity(phrase_vectors_matrix)

# Identify similar phrases sort them by similarity
similar_phrases = {}
for i, phrase in enumerate(phrases):
    similar_phrases[phrase] = [(phrases[j], similarity_matrix[i, j]) for j in np.argsort(similarity_matrix[i])[::-1] if j != i]

print('Time taken:', time.time() - st_time)

Time taken: 0.021220922470092773


In [67]:
# take user input and find similar phrases
user_input = "Is Buenos Aires flight cheap?"
# to vector
phrase_vectors = np.array([word2vec[word] for word in user_input.split() if word in word2vec])
# mean
phrase_vector = np.mean(phrase_vectors, axis=0)

# return the most similar phrase from the list of phrases by matrix multiplication
similarity = np.dot(phrase_vectors_matrix, phrase_vector)
most_similar_phrase = phrases[np.argmax(similarity)]
print(most_similar_phrase)

Airlines profitability Brazil?
